<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#4.-Inference" data-toc-modified-id="4.-Inference-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>4. Inference</a></span><ul class="toc-item"><li><span><a href="#4-1.-(just-for-test)-Data-Preparation" data-toc-modified-id="4-1.-(just-for-test)-Data-Preparation-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>4-1. (just for test) Data Preparation</a></span></li></ul></li></ul></div>

# 4. Inference

In [1]:
import sys, os
sys.path.append("../../")
sys.path.append("../../../")
sys.path.append("../../../../")
import setting
import pandas as pd

from Clust.clust.ML.common.common import p1_integratedDataSaving as p1
from Clust.clust.ML.tool import clean as ml_clean
from Clust.clust.ML.tool import data as ml_data
from Clust.clust.ML.tool import meta as ml_meta
from Clust.clust.ML.tool import model as ml_model
from Clust.clust.ML.tool import scaler as ml_scaler
import pathSetting

import torch
#import main_regression as mr
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device}" " is available.")

cpu is available.


## 4-1. (just for test) Data Preparation 

In [2]:
# 0. pick only one data
data_meta = p1.read_json_data(pathSetting.DataMetaPath)
dataList =  list(data_meta.keys())
# dataX

data_name_X = dataList[2]
data_save_mode_X = data_meta[data_name_X]["integrationInfo"]["DataSaveMode"]

# datay
data_name_y = dataList[3]
data_save_mode_y = data_meta[data_name_y]["integrationInfo"]["DataSaveMode"]
data_folder_name = "data_integrated_result"
current = os.getcwd()
data_folder_path = os.path.join(current, data_folder_name)

window_num =128
data_X = ml_data.get_saved_integrated_data(data_save_mode_X, data_name_X, data_folder_path)[:window_num]
# datay = p2.get_saved_integrated_data(dataSaveMode_y, dataName_y, dataFolderPath)[:1]
# from Clust.clust.transformation.type.DFToNPArray import transDFtoNP
# inputX, inputy = transDFtoNP(dataX, datay, windowNum)
# print(inputX, inputy) # Real Input

### Get model meta by mongodb

In [3]:
from Clust.clust.ingestion.mongo import mongo_client
from Clust.setting import influx_setting_KETI as ins

mongo_client = mongo_client.MongoClient(ins.CLUSTMetaInfo2)

db_name = 'model'
collection_name = 'meta'

all_model_meta = mongo_client.get_all_document(db_name, collection_name)

In [4]:
search = {'modelName': 'New_ClassificationNocleanLSTM_cf'}
model_meta= mongo_client.get_document_by_json(db_name, collection_name, search)
model_meta = model_meta[0]

In [5]:
model_meta

{'author': 'KETI',
 'modelName': 'New_ClassificationNocleanLSTM_cf',
 'modelVersion': 0.1,
 'trainDataInfo': {'startTime': '1992-1-11',
  'endTime': '2012-02-27',
  'dataInfo': [['bio_action_sensors', 'accelerationTrain']],
  'processParam': {'refine_param': {'removeDuplication': {'flag': False},
    'staticFrequency': {'flag': False, 'frequency': None}},
   'outlier_param': {'certainErrorToNaN': {'flag': False},
    'unCertainErrorToNaN': {'flag': False, 'param': {}}},
   'imputation_param': {'flag': False,
    'imputation_method': [],
    'totalNonNanRatio': 80}},
  'integration_freq_sec': 675,
  'cleanParam': 'NoClean',
  'DataSaveMode': 'CSV'},
 'featureList': ['col_0',
  'col_1',
  'col_2',
  'col_3',
  'col_4',
  'col_5',
  'col_6',
  'col_7',
  'col_8'],
 'target': ['value'],
 'trainDataType': 'timeseries',
 'modelPurpose': 'classification',
 'model_method': 'LSTM_cf',
 'modelTags': ['action', 'sensor', 'classification', 'pattern'],
 'cleanTrainDataParam': 'NoClean',
 'NaNProces

## Inference

In [6]:
X_scaler_file_path = model_meta['files']['XScalerFile']["filePath"]
y_scaler_file_path = model_meta['files']['yScalerFile']["filePath"]
model_file_path = model_meta['files']['modelFile']["filePath"]

feature_list = model_meta["featureList"]
target = model_meta["target"]
scaler_param = model_meta["scalerParam"]
model_method = model_meta["model_method"]
train_parameter = model_meta["trainParameter"]

dim = None
if model_method == "FC_cf":
    dim = 2

input_X, scaler_X = ml_scaler.get_scaled_test_data(data_X[feature_list], X_scaler_file_path, scaler_param)
# sacler_y = ml_scaler.get_scaler_file(y_scaler_file_path)

train_parameter['batch_size'] = 1
scaler_param="noScale" # for classification

In [7]:
from Clust.clust.ML.classification.inference import ClassificationInference

CInference = ClassificationInference()
CInference.set_param(train_parameter)
CInference.set_model(model_method, model_file_path)
CInference.set_data(input_X, window_num, dim)
preds =  CInference.inference()


Start inference

** Dimension of result for inference dataset = (1,)


In [8]:
if scaler_param =='scale':
    base_df_for_inverse = pd.DataFrame(columns=target, index=range(len(preds)))
    base_df_for_inverse[target] = preds
    prediction_result = pd.DataFrame(scaler_X.inverse_transform(base_df_for_inverse), columns=target, index=base_df_for_inverse.index)
else:
    prediction_result = pd.DataFrame(data={'value':preds}, index=range(len(preds)))

In [9]:
# df_result = app.get_inference_result(data_X, model_meta, window_num)

In [10]:
prediction_result

,value
0,4
